<a href="https://colab.research.google.com/github/MeenalShah13/SentimentAnalysisCOVID/blob/main/Sentiment_Classifier_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing of tweets

In [3]:
import pandas as pd
import numpy as np

In [ ]:
tweets=pd.read_csv('/covid19_tweets.csv')
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [ ]:
df = tweets

In [ ]:
df.shape

(179108, 13)

In [ ]:
df.isnull().sum() 

user_name               0
user_location       36771
user_description    10286
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
hashtags            51334
source                 77
is_retweet              0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False)

In [ ]:
df.isnull().sum()

user_name           0
user_location       0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
dtype: int64

In [ ]:
df.shape

(99138, 13)

## Sampling

In [ ]:
sample = df.sample(n=15000)
#sample.to_csv('sample_data.csv',index=False)

In [ ]:
sample.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
31128,John McPherson🏴󠁧󠁢󠁳󠁣󠁴󠁿⚽️🇺🇸,"Arizona, USA",Scottish (naturalized American). Amazing wife ...,2008-08-01 19:31:48,833,532,55861,False,2020-07-27 02:25:52,"That's something like 150,000 per person who d...",['COVID19'],Twitter for Android,False
40805,PETER MAER,"Washington, DC",Retired White House Corr. \nEdward R. Murrow A...,2010-03-12 15:20:56,13370,11483,1538,True,2020-07-29 16:15:48,As Trump continues to talk up #hydroxycholoroq...,"['hydroxycholoroquine', 'DrFauci']",Twitter for iPhone,False
142424,The Korea Herald,"Seoul, Korea","코리아헤럴드 Established in 1953, Korea's No.1 Engli...",2009-07-13 01:58:14,65722,25,32,True,2020-08-17 07:04:52,Culture sector scrambles as COVID-19 cases soa...,"['SouthKorea', 'culture', 'COVID19', 'pandemic']",Twitter Web App,False
132768,Andrew Davis,"London, England","A personal view of life, music and Hong Kong n...",2009-12-11 08:52:51,2715,4186,64387,False,2020-08-16 09:17:53,End of an era: has #coronavirus killed the #US...,"['coronavirus', 'US', 'Democrats', 'Republican...",Twitter for iPhone,False
153865,OrgThis Blog - S.Lee,"Texas, USA","Blog for organization, productivity, and style...",2011-02-28 20:15:47,10479,11742,0,False,2020-08-18 16:19:00,How to Clean Your Reusable #facemasks https://...,"['facemasks', 'Tips', 'Video', 'Covid19']",TweetDeck,False


In [ ]:
df1 = sample.drop(['user_created','user_description','user_followers','user_friends','user_favourites','user_verified','source','is_retweet'],axis=1)
df1.head()

,user_name,user_location,date,text,hashtags
31128,John McPherson🏴󠁧󠁢󠁳󠁣󠁴󠁿⚽️🇺🇸,"Arizona, USA",2020-07-27 02:25:52,"That's something like 150,000 per person who d...",['COVID19']
40805,PETER MAER,"Washington, DC",2020-07-29 16:15:48,As Trump continues to talk up #hydroxycholoroq...,"['hydroxycholoroquine', 'DrFauci']"
142424,The Korea Herald,"Seoul, Korea",2020-08-17 07:04:52,Culture sector scrambles as COVID-19 cases soa...,"['SouthKorea', 'culture', 'COVID19', 'pandemic']"
132768,Andrew Davis,"London, England",2020-08-16 09:17:53,End of an era: has #coronavirus killed the #US...,"['coronavirus', 'US', 'Democrats', 'Republican..."
153865,OrgThis Blog - S.Lee,"Texas, USA",2020-08-18 16:19:00,How to Clean Your Reusable #facemasks https://...,"['facemasks', 'Tips', 'Video', 'Covid19']"


In [ ]:
df1['user_location'].value_counts()

India                       453
United States               255
New Delhi, India            235
Mumbai, India               181
Washington, DC              138
                           ... 
Turn On Notifications 📲🔔      1
Lahaska, PA                   1
ÜT: 18.003518,-76.788264      1
Graz, Österreich              1
Mobile:0727-444454            1
Name: user_location, Length: 5634, dtype: int64

In [ ]:
df1.shape

(15000, 5)

In [ ]:
df1.to_csv('sampledata.csv',index=False)

## Sentiment Polarity


In [ ]:
df1 = pd.read_csv("sampledata.csv")

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
blanks = []  # start with an empty list

for i,un,ul,d,t,h in df1.itertuples():  # iterate over the DataFrame
    if type(t)==str:            # avoid NaN values
        if t.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df1.drop(blanks, inplace=True)
df1.shape

(15000, 5)

In [ ]:
df1['scores'] = df1['text'].apply(lambda review: sid.polarity_scores(review))

df1['compound']  = df1['scores'].apply(lambda score_dict: score_dict['compound'])


df1['label'] = df1['compound'].apply(lambda c: 'pos' if c >0 else ('neu' if c==0 else 'neg'))


df1.head()

,user_name,user_location,date,text,hashtags,scores,compound,label
0,John McPherson🏴󠁧󠁢󠁳󠁣󠁴󠁿⚽️🇺🇸,"Arizona, USA",2020-07-27 02:25:52,"That's something like 150,000 per person who d...",['COVID19'],"{'neg': 0.138, 'neu': 0.766, 'pos': 0.096, 'co...",-0.2732,neg
1,PETER MAER,"Washington, DC",2020-07-29 16:15:48,As Trump continues to talk up #hydroxycholoroq...,"['hydroxycholoroquine', 'DrFauci']","{'neg': 0.097, 'neu': 0.903, 'pos': 0.0, 'comp...",-0.1027,neg
2,The Korea Herald,"Seoul, Korea",2020-08-17 07:04:52,Culture sector scrambles as COVID-19 cases soa...,"['SouthKorea', 'culture', 'COVID19', 'pandemic']","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
3,Andrew Davis,"London, England",2020-08-16 09:17:53,End of an era: has #coronavirus killed the #US...,"['coronavirus', 'US', 'Democrats', 'Republican...","{'neg': 0.201, 'neu': 0.67, 'pos': 0.129, 'com...",-0.3818,neg
4,OrgThis Blog - S.Lee,"Texas, USA",2020-08-18 16:19:00,How to Clean Your Reusable #facemasks https://...,"['facemasks', 'Tips', 'Video', 'Covid19']","{'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'comp...",0.4019,pos


In [ ]:
df1.to_csv('finalsample.csv',index=False)

In [ ]:
df1['label'].value_counts()

neu    5701
pos    5438
neg    3861
Name: label, dtype: int64

## Contractions removal

In [ ]:
import re
contractions_dict = {"can`t":"can not",
                     "won`t":"will not",
                     "don`t":"do not",
                     "aren`t":"are not",
                     "i`d":"i would",
                     "couldn`t": "could not",
                     "shouldn`t": "should not",
                     "wouldn`t": "would not",
                     "isn`t": "is not",
                     "it`s": "it is",
                     "didn`t": "did not",
                     "weren`t": "were not",
                     "mustn`t": "must not",
                    }

def prepare_data(df1:pd.DataFrame) -> pd.DataFrame:
    
    df1["text"] = df1["text"] \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))
        
    df1["label"] = df1["label"].map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
    return df1.text.values, df1.label.values
def replace_words(string:str, dictionary:dict):
    for k, v in dictionary.items():
        string = string.replace(k, v)
    return string


##Lowercasing, Removing stopwords, and lemmatization

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"

for words in range(len(df1["text"])):
  word_tokens = word_tokenize(df1["text"][words].lower())
  filtered_sentence = [w for w in word_tokens if not w in stop_words]
  filtered_sentence = []
  for w in word_tokens:
    if w not in stop_words and w not in symbols:
      filtered_sentence.append(lemmatizer.lemmatize(w))
  df1["text"][words] = " ".join(filtered_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Random forest classifier

In [ ]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

X = df1.text
Y =df1.label

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



X_train = X_train \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

X_test = X_test \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

y_train =  y_train.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
y_test = y_test.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )


(12000,) (12000,)
(3000,) (3000,)


In [ ]:
train_tweets, train_labels = X_train,y_train
test_tweets, test_labels = X_test,y_test

In [ ]:
pip install keras


_______________________________________

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tweets)
train_tokenized = tokenizer.texts_to_matrix(
                             train_tweets,
                             mode='tfidf'
                             )
test_tokenized = tokenizer.texts_to_matrix(
                             test_tweets,
                             mode='tfidf'
                             )

In [ ]:
from  sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(
                                n_estimators=500, 
                                min_samples_leaf=2,
                                oob_score=True,
                                n_jobs=-1,
                                )
forest.fit(train_tokenized,train_labels)
print(f"Train score: {forest.score(train_tokenized,train_labels)}")
print(f"OOB score: {forest.oob_score_}")

Train score: 0.8415833333333333
OOB score: 0.7650833333333333


In [ ]:
print("Test score:", forest.score(test_tokenized,test_labels))

Test score: 0.756
